In [48]:
import re
import glob
import nltk
import numpy as np
import pandas as pd
from pprint import pprint

In [89]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.matutils import cossim

In [7]:
import spacy

In [12]:
import pyLDAvis
import pyLDAvis.gensim

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
docs = glob.glob("./result/clean_tweets/*.csv")

In [70]:
frames = []
for doc in docs:
    df = pd.read_csv(doc, sep='\t')
    frames.append(df)

In [71]:
result = pd.concat(frames)

In [141]:
result.shape

(3073659, 4)

In [77]:
dates = {}
for i in range(result.shape[0]):
    date = result["created_at"].iloc[i]
    year, month, day = date.split("-")
    
    if year not in dates:
        dates[year] = {}
    
    if month not in dates[year]:
        dates[year][month] = {}
    
    if day not in dates[year][month]:
        dates[year][month][day] = 0
    
    dates[year][month][day] += 1

In [17]:
df = pd.read_csv(doc, sep='\t')

In [31]:
x = df["cleaned_text"].loc[0].split()

In [137]:
x

['@pamonshaw', '@ninawmakeup', 'nic', 'mem']

In [34]:
xs = [df["cleaned_text"].loc[i].split() for i in range(df.shape[0])]

In [138]:
def word_pairs(sen):
    items = sen
    return ["_".join([items[i],items[j]]) for i in range(len(items)) for j in range(i+1, len(items))]

In [139]:
word_pairs(x)

['@pamonshaw_@ninawmakeup',
 '@pamonshaw_nic',
 '@pamonshaw_mem',
 '@ninawmakeup_nic',
 '@ninawmakeup_mem',
 'nic_mem']

In [37]:
id2word = corpora.Dictionary(xs)

In [39]:
texts = xs
corpus = [id2word.doc2bow(text) for text in texts]

In [40]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [44]:
id2word[3]

'nic'

In [47]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [86]:
lda_model.print_topics()

[(0,
  '0.157*"lip" + 0.119*"colo" + 0.102*"skin" + 0.058*"ton" + 0.045*"light" + 0.037*"pink" + 0.031*"brown" + 0.029*"want" + 0.021*"gold" + 0.014*"might"'),
 (1,
  '0.267*"xo" + 0.231*"ye" + 0.168*"day" + 0.064*"@littlethingsto" + 0.030*"@tserings" + 0.014*"@gailsiemko" + 0.009*"@catlovertamjo" + 0.009*"tamr" + 0.005*"gail" + 0.000*"christmas"'),
 (2,
  '0.561*"@ninawmakeup" + 0.056*"canad" + 0.049*"nic" + 0.033*"agr" + 0.025*"roug" + 0.020*"get" + 0.019*"pretty" + 0.016*"hug" + 0.016*"@panagiota_karag" + 0.013*"neut"'),
 (3,
  '0.407*"thank" + 0.256*"gre" + 0.121*"much" + 0.054*"perfect" + 0.015*"gift" + 0.000*"holiday" + 0.000*"christmas" + 0.000*"merry" + 0.000*"2019" + 0.000*"glad"'),
 (4,
  '0.397*"good" + 0.287*"morn" + 0.047*"luck" + 0.004*"@irenek24" + 0.004*"ir" + 0.000*"thursday" + 0.000*"enjoy" + 0.000*"friend" + 0.000*"may" + 0.000*"dear"'),
 (5,
  '0.322*"lol" + 0.193*"look" + 0.067*"see" + 0.040*"week" + 0.014*"launch" + 0.011*"forward" + 0.001*"davelackiebeauty" + 0.0

In [53]:
doc_lda = lda_model[corpus]

In [54]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -16.061567172264496


In [56]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4744900323380958


In [57]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/marksho/miniconda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.328312 -0.310861       1        1  11.531892
18    -0.149446  0.161246       2        1  10.896242
2     -0.174730  0.242183       3        1   8.494866
17    -0.091897  0.049884       4        1   8.138381
3     -0.100933  0.058274       5        1   7.970460
10    -0.014508  0.024527       6        1   7.041679
14    -0.059092  0.023916       7        1   6.693573
1     -0.003141 -0.003843       8        1   4.877930
11     0.058848 -0.021824       9        1   4.670788
4      0.045028 -0.017750      10        1   4.180712
16    -0.028353  0.014606      11        1   4.168396
9      0.084792 -0.025339      12        1   3.831026
12     0.053794 -0.020524      13        1   3.737093
5      0.088472 -0.025616      14        1   3.602672
15     0.093970 -0.026560      15        1   3.329779
7      0.064605 -0.022541      16        1   3.273675
0      0.097271 -0.026769      17        1   2.761381
19     0.121518 -0.025646      18        1   0.471145
6      0.121078 -0.023706      19        1   0.193257
13     0.121036 -0.023655      20        1   0.135064, topic_info=     Category         Freq              Term        Total  loglift  logprob
7     Default  1226.000000             happy  1226.000000  30.0000  30.0000
0     Default  1127.000000      @ninawmakeup  1127.000000  29.0000  29.0000
32    Default   768.000000             thank   768.000000  28.0000  28.0000
83    Default   633.000000               hop   633.000000  27.0000  27.0000
143   Default   394.000000              good   394.000000  26.0000  26.0000
36    Default   483.000000               gre   483.000000  25.0000  25.0000
232   Default   372.000000               new   372.000000  24.0000  24.0000
108   Default   296.000000              real   296.000000  23.0000  23.0000
166   Default   309.000000                xo   309.000000  22.0000  22.0000
10    Default   275.000000               lol   275.000000  21.0000  21.0000
207   Default   428.000000              wish   428.000000  20.0000  20.0000
43    Default   416.000000               lov   416.000000  19.0000  19.0000
202   Default   285.000000              morn   285.000000  18.0000  18.0000
5     Default   411.000000            beauty   411.000000  17.0000  17.0000
42    Default   268.000000                ye   268.000000  16.0000  16.0000
346   Default   278.000000               lot   278.000000  15.0000  15.0000
101   Default   204.000000               fun   204.000000  14.0000  14.0000
11    Default   303.000000       @davelackie   303.000000  13.0000  13.0000
8     Default   257.000000               lik   257.000000  12.0000  12.0000
325   Default   280.000000              wond   280.000000  11.0000  11.0000
205   Default   195.000000               day   195.000000  10.0000  10.0000
97    Default   172.000000              best   172.000000   9.0000   9.0000
170   Default   229.000000              much   229.000000   8.0000   8.0000
106   Default   165.000000              look   165.000000   7.0000   7.0000
25    Default   202.000000                on   202.000000   6.0000   6.0000
197   Default   154.000000               com   154.000000   5.0000   5.0000
171   Default   139.000000               nam   139.000000   4.0000   4.0000
209   Default   132.000000             congr   132.000000   3.0000   3.0000
275   Default   126.000000              hear   126.000000   2.0000   2.0000
210   Default   127.000000              drew   127.000000   1.0000   1.0000
...       ...          ...               ...          ...      ...      ...
1642  Topic20     0.009971         @jmcgratj     1.423860   1.6457  -8.0718
1643  Topic20     0.009971         @114leese     1.423854   1.6457  -8.0718
1645  Topic20     0.009971  @lalalaurabingle     1.423854   1.6457  -8.0718
1647  Topic20     0.009971            confid     1.423854   1.6457  -8.0718
3202  Topic20     0.009971       

In [87]:
lda_model.show_topic(0)

[('lip', 0.15747344),
 ('colo', 0.11921731),
 ('skin', 0.10162249),
 ('ton', 0.05793143),
 ('light', 0.04516961),
 ('pink', 0.03723994),
 ('brown', 0.03123095),
 ('want', 0.02937598),
 ('gold', 0.020900657),
 ('might', 0.014313835)]

In [95]:
cossim(lda_model.show_topic(0), lda_model.show_topic(0))

0.9999999999999999

In [97]:
dates.keys()

dict_keys(['2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008'])

In [100]:
dates["2018"]["04"]

{'30': 2387,
 '27': 2235,
 '26': 2385,
 '24': 2505,
 '23': 2565,
 '21': 1544,
 '15': 1620,
 '14': 1602,
 '13': 2336,
 '12': 2261,
 '08': 1445,
 '04': 2069,
 '29': 1533,
 '28': 1537,
 '25': 2287,
 '22': 1476,
 '20': 2380,
 '19': 2384,
 '18': 2396,
 '17': 2260,
 '16': 2261,
 '11': 2428,
 '10': 2318,
 '09': 2279,
 '07': 1372,
 '06': 2090,
 '05': 2119,
 '03': 2057,
 '02': 1803,
 '01': 1315}

In [119]:
def train_lda(df, num_topics=20):
    df = df.dropna()
    xs = [df["cleaned_text"].iloc[i].split() for i in range(df.shape[0])]
    
    id2word = corpora.Dictionary(xs)
    texts = xs
    corpus = [id2word.doc2bow(text) for text in texts]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    return lda_model

In [120]:
doc = result[result["created_at"] == "2018-04-30"]
lda_2018_04_30 = train_lda(doc)

In [121]:
doc = result[result["created_at"] == "2018-04-27"]
lda_2018_04_27 = train_lda(doc)

In [128]:
topics = {}
for i in range(20):
    vec_1 = lda_2018_04_30.show_topic(i)
    
    top_sim = -1
    for j in range(20):
        vec_2 = lda_2018_04_27.show_topic(j)
        sim = cossim(vec_1, vec_2)
        if sim > top_sim:
            top_sim = sim
            topics[i] = (j, sim)

In [ ]:
from gensim.test.utils import datapath

In [154]:
topic_model_dates = []
topic_models = []
for i, year in enumerate(dates.keys()):
    if int(year) < 2016:
        continue
    for j, month in enumerate(dates[year].keys()):
        for k, day in enumerate(dates[year][month].keys()):
            print(i, j / len(dates[year].keys()), k / len(dates[year][month].keys()))
            date = "{}-{}-{}".format(year, month, day)
            doc = result[result["created_at"] == date]
            model = train_lda(doc)
            
            temp_file = datapath(date)
            lda.save(temp_file)
            
            topic_model_dates.append(date)
            topic_models.append(model)

0 0.0 0.0
0 0.0 0.14285714285714285
0 0.0 0.2857142857142857
0 0.0 0.42857142857142855
0 0.0 0.5714285714285714
0 0.0 0.7142857142857143
0 0.0 0.8571428571428571
0 0.5 0.0
0 0.5 0.03225806451612903


KeyboardInterrupt: 

In [ ]:
def cossim_pairs(topic_models, num_topics=20):
    topic_cos_map = {}
    
    for i, m in enumerate(topic_models):
        for u in range(num_topics):
            for j, n in enumerate(topic_models):
                if i == j:
                    continue
                top_cs = -1
                top_topic = ""
                for v in range(num_topics):
                    cs = cossim(m.show_topic(u), n.show_topic(v))
                    if cs > top_cs:
                        top_cs = cs
                        top_topic = "{}:{}_{}:{}".format(i, u, j, v)
                
                topic_cos_map[top_topic] = top_cs